# 01 — Data Quality & Cleaning Pipeline
---

This notebook loads the NovaCred credit application JSON dataset, performs a data quality audit, and applies remediation steps to produce analysis-ready datasets.

**Input:** `data/raw/raw_credit_applications.json` 

**Outputs:**  
- `data/processed/applications_clean.csv` (cleaned, may still contain PII)  
- `data/processed/applications_analysis_ready.csv` (PII removed for analysis)

The notebook covers duplicate records, missing/incomplete values, inconsistent data types, inconsistent categorical coding, inconsistent date formats, invalid/impossible values, and internal consistency checks.

In [713]:
# Imports
import json
import warnings
import ipaddress
from pathlib import Path

import numpy as np
import pandas as pd

---
## 0. Load & schema sanity check
---
Before running any quality checks, we first confirm the data loaded correctly, checking the dataset shape, previewing a few rows, and verifying that the key fields we expect from the schema are present.

In [714]:
# Raw file path
data_path = Path("../data/raw/raw_credit_applications.json")

# Read JSON
with data_path.open("r", encoding="utf-8") as f:
    apps = json.load(f)

# Flatten nested JSON into a tabular DataFrame
df_raw = pd.json_normalize(apps, sep=".")

# Basic sanity checks
print("rows, cols:", df_raw.shape)
display(df_raw.head(3))

# Check expected columns
expected = {
    "_id",
    "spending_behavior",
    "processing_timestamp",
    "applicant_info.full_name",
    "applicant_info.ssn",
    "financials.annual_income",
    "financials.credit_history_months",
    "financials.debt_to_income",
    "financials.savings_balance",
    "decision.loan_approved",
}

missing = expected - set(df_raw.columns)
print("Missing expected columns:", missing)

rows, cols: (502, 21)


,_id,spending_behavior,processing_timestamp,applicant_info.full_name,applicant_info.email,applicant_info.ssn,applicant_info.ip_address,applicant_info.gender,applicant_info.date_of_birth,applicant_info.zip_code,...,financials.credit_history_months,financials.debt_to_income,financials.savings_balance,decision.loan_approved,decision.rejection_reason,loan_purpose,decision.interest_rate,decision.approved_amount,financials.annual_salary,notes
0,app_200,"[{'category': 'Shopping', 'amount': 480}, {'ca...",2024-01-15T00:00:00Z,Jerry Smith,jerry.smith17@hotmail.com,596-64-4340,192.168.48.155,Male,2001-03-09,10036,...,23,0.20,31212,False,algorithm_risk_score,NaN,NaN,NaN,NaN,NaN
1,app_037,"[{'category': 'Rent', 'amount': 608}, {'catego...",NaN,Brandon Walker,brandon.walker2@yahoo.com,425-69-4784,10.1.102.112,M,1992-03-31,10032,...,51,0.18,17915,False,algorithm_risk_score,NaN,NaN,NaN,NaN,NaN
2,app_215,"[{'category': 'Rent', 'amount': 109}]",NaN,Scott Moore,scott.moore94@mail.com,370-78-5178,10.240.193.250,Male,1989-10-24,10075,...,41,0.21,37909,True,NaN,vacation,3.7,59000.0,NaN,NaN


Missing expected columns: set()


The dataset was loaded successfully (502 rows, 21 columns) and all key expected columns are present.

---
## 1. Data quality checks: `spending_behavior`
---

`spending_behavior` is a nested list of spending entries (each entry has a category and an amount). Since it’s an “array of objects”, we can’t treat it like a normal column, meaning we need to unpack it first so we can run data quality checks. After that, we check missing/incomplete entries, consistent data types, valid values, and consistent category formatting.

### 1A. Create an analysis table

To check quality at the entry level, we explode the list so each spending entry becomes its own row. This gives us a clean table (`spend`) where each row is one item (`category`, `amount`)  linked to the original application.

In [715]:
# Build an exploded table: one row per spending entry
spend = (
    df_raw[["_id", "spending_behavior"]]
    .dropna(subset=["spending_behavior"])
    .explode("spending_behavior", ignore_index=True)
)

# Extract fields safely
spend["category"] = spend["spending_behavior"].map(lambda x: x.get("category") if isinstance(x, dict) else None)
spend["amount"] = pd.to_numeric(
    spend["spending_behavior"].map(lambda x: x.get("amount") if isinstance(x, dict) else None),
    errors="coerce"
)

print("Total spending entries:", len(spend))
display(spend[["_id", "category", "amount"]].head(10))

Total spending entries: 827


,_id,category,amount
0,app_200,Shopping,480
1,app_200,Rent,790
2,app_200,Alcohol,247
3,app_037,Rent,608
4,app_037,Dining,96
5,app_037,Healthcare,243
6,app_215,Rent,109
7,app_024,Fitness,575
8,app_184,Entertainment,463
9,app_275,Entertainment,571


### 1B. Missing/incomplete entries

**Dimension:** Completeness

Here we’re just checking if anything is missing in spending_behavior. First, we check if any applications have spending_behavior missing entirely. Then, once we explode the list into one row per spending entry, we check at the entry level to make sure every entry actually has both fields filled in (category and amount).

In [716]:
# Total number of rows in the main table
n_apps = len(df_raw)

# Count applications where spending_behavior is missing
missing_sb_apps = df_raw["spending_behavior"].isna().sum()
print("Applications missing spending_behavior:", missing_sb_apps, f"({missing_sb_apps/n_apps*100:.2f}%)")

# Total number of spending entries (rows) after exploding the array
n_entries = len(spend)

# Count entries where category is missing
missing_cat = spend["category"].isna().sum()

# Count entries where amount is missing
missing_amt = spend["amount"].isna().sum()

print("Missing category entries:", missing_cat, f"({missing_cat/max(n_entries,1)*100:.2f}%)")
print("Missing amount entries:", missing_amt, f"({missing_amt/max(n_entries,1)*100:.2f}%)")

Applications missing spending_behavior: 0 (0.00%)
Missing category entries: 0 (0.00%)
Missing amount entries: 0 (0.00%)


`spending_behavior` has 0 missing arrays and 0 entries missing `category` or `amount`.

### 1C. Inconsistent data types

**Dimension:** Consistency

Now we check if every `category` is a string and every `amount` is a number.

In [717]:
# Count values that aren't strings
non_string_cat = (~spend["category"].dropna().map(lambda x: isinstance(x, str))).sum()
print("Non-string category entries:", non_string_cat,
      f"({non_string_cat/max(n_entries, 1)*100:.2f}%)")

# Count amounts that are missing or not numeric
non_numeric_amount = spend["amount"].isna().sum()
print("Non-numeric/missing amount entries:", non_numeric_amount,
      f"({non_numeric_amount/max(n_entries, 1)*100:.2f}%)")


Non-string category entries: 0 (0.00%)
Non-numeric/missing amount entries: 0 (0.00%)


All `spending_behavior` entries have consistent types. 

### 1D. Invalid values

**Dimensions:** Validity

Check for impossible values, in this case, negative spending amounts.

In [718]:
# Count negative amounts
neg_amount = int((spend["amount"] < 0).sum())

print("Negative amount entries:", neg_amount, f"({neg_amount/max(n_entries,1)*100:.2f}%)")

Negative amount entries: 0 (0.00%)


There are no invalid spending amounts. 

### 1E. Category formatting consistency

**Dimension:** Consistency

We check whether category values are consistently written, meaning if they have the same capitalization and spacing. We compare the raw categories to a normalized version (trimmed + lowercase) to see if multiple raw spellings map to the same category.

In [719]:
# Normalize categories: trim + lowercase
raw = spend["category"].astype("string")
norm = raw.str.strip().str.lower()

print("Distinct raw categories:", raw.dropna().nunique())
print("Distinct normalized categories:", norm.dropna().nunique())

# Show categories that only differ by case/spacing (if any)
variants = raw.groupby(norm).unique()
variants = variants[variants.map(len) > 1]

if variants.empty:
    print("No case/spacing variants found.")
else:
    display(variants)

Distinct raw categories: 15
Distinct normalized categories: 15
No case/spacing variants found.


The categories are already consistently formatted (15 raw = 15 normalized), so we found no case or spacing variants to standardize.

### 1F. Key findings
After unpacking the list into an entry-level table, we found no missing categories/amounts, no non-numeric or negative amounts, and no category formatting inconsistencies. Overall, spending_behavior looks clean and consistent and doesn't require remediation.

---
## 2. Data quality checks: Main table
---

The main table has one row per application, so we check data quality at the application level. Here we focus on the core fields (applicant info, financials, decision) and look for the data quality issues mentioned previously. 

### 2A. Duplicate records 

**Dimension:** Uniqueness

Each application in the dataset should be unique, so we check for duplicates using the two identifiers: duplicate application IDs (`_id`) and duplicate applicant Social Security Numbers (`applicant_info.ssn`, excluding missing values)

In [720]:
# Total number of applications
n = len(df_raw)

# ID duplicates
dup_id = df_raw["_id"].duplicated().sum()

# SSN duplicates (excluding missing)
ssn = df_raw["applicant_info.ssn"]
missing_ssn = ssn.isna().sum()
dup_ssn_excl_missing = ssn.dropna().duplicated().sum()
n_non_missing_ssn = ssn.notna().sum()

print("Duplicate _id:", dup_id, f"({dup_id/n*100:.2f}%)")
print("Missing SSN:", missing_ssn, f"({missing_ssn/n*100:.2f}%)")
print("Duplicate SSN (excluding missing):", dup_ssn_excl_missing,
      f"({dup_ssn_excl_missing/max(n_non_missing_ssn,1)*100:.2f}% of non-missing SSNs)")

Duplicate _id: 2 (0.40%)
Missing SSN: 5 (1.00%)
Duplicate SSN (excluding missing): 3 (0.60% of non-missing SSNs)


Results show 2 duplicate application IDs (0.40%), and 3 duplicate SSNs among non-missing values (0.60%), indicating uniqueness issues.

In [721]:
# Show duplicated _id rows
dup_id_rows = df_raw[df_raw["_id"].duplicated(keep=False)].sort_values("_id")
display(dup_id_rows[["_id", "applicant_info.full_name", "applicant_info.ssn", "processing_timestamp"]])

# Show duplicated SSN rows (excluding missing SSNs)
dup_ssn_rows = df_raw[df_raw["applicant_info.ssn"].notna() & df_raw["applicant_info.ssn"].duplicated(keep=False)] \
    .sort_values("applicant_info.ssn")
display(dup_ssn_rows[["_id", "applicant_info.full_name", "applicant_info.ssn"]].head(20))

,_id,applicant_info.full_name,applicant_info.ssn,processing_timestamp
383,app_001,Stephanie Nguyen,427-90-1892,NaN
455,app_001,Stephanie Nguyen,NaN,NaN
8,app_042,Joseph Lopez,652-70-5530,NaN
354,app_042,Joseph Lopez,652-70-5530,NaN


,_id,applicant_info.full_name,applicant_info.ssn
8,app_042,Joseph Lopez,652-70-5530
354,app_042,Joseph Lopez,652-70-5530
92,app_088,Susan Martinez,780-24-9300
122,app_016,Gary Wilson,780-24-9300
16,app_101,Sandra Smith,937-72-8731
499,app_234,Samuel Hill,937-72-8731


These tables show that the `_id` duplicates appear as repeated application records, and the SSN duplicates appear either within the same `_id` (`app_042`) or across different `_id`s (e.g., the same SSN linked to multiple applications). The latter appear in more than one record with different applicant names, which is an accuracy/cross-field consistency issue. We treat these as SSN collisions.

In remediation, we deduplicate `_id` (keeping the most complete record) and flag the remaining SSN duplicates instead of assuming which record is correct.

### 2B. Missing / incomplete records 

**Dimension:** Completeness

#### 2B.1 Missing values

We report both missing counts and missing percentages at the application level. Some fields are expected to be conditionally missing, such as rejection reasons, which only exist for denied applications, so we interpret missingness in context.

In [722]:
# Count missing values per column
missing_count = df_raw.isna().sum().sort_values(ascending=False)

# Percent missing per column
missing_pct = (df_raw.isna().mean() * 100).round(2).sort_values(ascending=False)

# Combine into one table for reporting
missing_table = pd.DataFrame({
    "missing_count": missing_count,
    "missing_%": missing_pct
})

# Only keep columns with at least 1 missing value
display(missing_table[missing_table["missing_count"] > 0])

,missing_count,missing_%
notes,500,99.60
financials.annual_salary,497,99.00
loan_purpose,452,90.04
processing_timestamp,440,87.65
decision.rejection_reason,292,58.17
decision.approved_amount,210,41.83
decision.interest_rate,210,41.83
financials.annual_income,5,1.00
applicant_info.ip_address,5,1.00
applicant_info.ssn,5,1.00


The most incomplete fields are `notes` (99.6%), `financials.annual_salary` (99.0%), `loan_purpose` (90.0%), and `processing_timestamp` (87.7%).
Additionally, several decision-related fields appear conditionally missing, which is expected:
  - `decision.approved_amount` and `decision.interest_rate` are missing for 41.83% of records (consistent with applications that were not approved).
  - `decision.rejection_reason` is missing for 58.17% of records (consistent with approved applications).

The dataset includes both `financials.annual_income` and `financials.annual_salary`, which represent the same underlying concept (yearly income) but are stored as two different variables. Since `annual_salary` is almost always missing, this indicates inconsistent field usage and can confuse downstream analysis.

To remediate this situation, we will merge `annual_salary` into `annual_income` whenever `annual_income` is missing but `annual_salary` is available, and then drop `annual_salary` to enforce a single canonical income field.


#### 2B.2 Blank entries

Missingness checks using isna() only capture true NaN values. However, several fields also use blank strings, like "" or " ", to represent “missing”, which would be invisible in the standard missing-value table.

Here we quantify blank/empty-string values across all text-like columns to get a more accurate picture of completeness.

In [723]:
# Count blank/empty-string values per column
s = df_raw.select_dtypes(include=["object", "string"]).copy()

# Count values that are empty after trimming spaces
blank_count = s.apply(lambda col: col.astype("string").str.strip().eq("").sum()).sort_values(ascending=False)

# Convert blank counts into percentages of total rows
blank_pct   = (blank_count / len(df_raw) * 100).round(2)

# Combine into a reporting table
blank_table = pd.DataFrame({
    "blank_count": blank_count,
    "blank_%": blank_pct
})

# Show only columns that actually have blanks
display(blank_table[blank_table["blank_count"] > 0])

,blank_count,blank_%
applicant_info.email,7,1.39
applicant_info.date_of_birth,4,0.80
applicant_info.gender,2,0.40
applicant_info.zip_code,1,0.20


The table shows that a small share of records contain blanks entries. 

In cleaning, we will treat blanks as missing values before any further validation or analysis.

### 2C. Inconsistent data types 

**Dimension:** Consistency

#### 2C.1 Numeric fields

In this step, we verify that fields expected to be numeric are consistently stored as numeric values. Type inconsistencies can break aggregations, distort summary statistics, and cause issues in downstream modelling, so we identify mixed-type columns before remediation.

In [724]:
numeric_cols = [
    "financials.annual_income",
    "financials.annual_salary",
    "financials.credit_history_months",
    "financials.debt_to_income",
    "financials.savings_balance",
    "decision.interest_rate",
    "decision.approved_amount",
]

# For each numeric column, count how many values are int/float/str (excluding NaN)
type_counts = {}
for col in numeric_cols:
    s = df_raw[col]
    counts = s.dropna().map(lambda x: type(x).__name__).value_counts()
    type_counts[col] = counts.to_dict()

# Convert the dictionary into a table
type_counts_df = pd.DataFrame(type_counts).fillna(0).astype(int).T

print("Type counts by column (non-missing values):")
display(type_counts_df)

# Percentages (out of non-missing values in each column)
non_missing = df_raw[numeric_cols].notna().sum()
type_pct_df = (type_counts_df.div(non_missing, axis=0) * 100).round(2)

print("Type percentages by column (non-missing values):")
display(type_pct_df)

Type counts by column (non-missing values):


,int,str,float
financials.annual_income,488,8,1
financials.annual_salary,0,0,5
financials.credit_history_months,502,0,0
financials.debt_to_income,0,0,502
financials.savings_balance,502,0,0
decision.interest_rate,0,0,292
decision.approved_amount,0,0,292


Type percentages by column (non-missing values):


,int,str,float
financials.annual_income,98.19,1.61,0.2
financials.annual_salary,0.00,0.00,100.0
financials.credit_history_months,100.00,0.00,0.0
financials.debt_to_income,0.00,0.00,100.0
financials.savings_balance,100.00,0.00,0.0
decision.interest_rate,0.00,0.00,100.0
decision.approved_amount,0.00,0.00,100.0


`financials.annual_income` shows mixed types: most values are numeric, but 8 records store income as strings (and 1 as float), indicating inconsistent typing across records.

The other fields are consistently numeric. In remediation, we will convert income fields to numeric


#### 2C.2 String and boolean fields

As a quick schema validation step, we verify that fields expected to be strings contain only string values (for non-missing entries), and that `decision.loan_approved` behaves as a boolean. 

In [725]:
# Columns that should be stored as strings 
expected_str = [
    "_id",
    "applicant_info.full_name",
    "applicant_info.email",
    "applicant_info.ssn",
    "applicant_info.ip_address",
    "applicant_info.gender",
    "applicant_info.date_of_birth",
    "applicant_info.zip_code",
    "decision.rejection_reason",
    "loan_purpose",
    "notes",
]

# Count how many non-missing values aren't strings per column
non_string_counts = {}
for col in expected_str:
    if col in df_raw.columns:
        s = df_raw[col]
        non_string_counts[col] = int(s.dropna().map(lambda x: not isinstance(x, str)).sum())

# Show counts
display(pd.Series(non_string_counts).sort_values(ascending=False))

_id                             0
applicant_info.full_name        0
applicant_info.email            0
applicant_info.ssn              0
applicant_info.ip_address       0
applicant_info.gender           0
applicant_info.date_of_birth    0
applicant_info.zip_code         0
decision.rejection_reason       0
loan_purpose                    0
notes                           0
dtype: int64

In [726]:
# Boolean check for loan_approved
vals = df_raw["decision.loan_approved"].dropna().unique()
print("Unique values:", vals)

# Confirm all observed values are boolean
is_bool = all(isinstance(v, (bool, np.bool_)) for v in vals)
print("All boolean?", is_bool)

Unique values: [False  True]
All boolean? True


All fields expected to be strings contain only string values for non-missing entries and `decision.loan_approved` contains only boolean values, so these fields are type-consistent with the schema.

### 2D. Inconsistent coding/formatting of categorical fields

**Dimension:** Consistency  

Categorical fields should use consistent representations (e.g., avoid mixing `Male` vs `M`, casing/spacing variants, and blanks). 

#### 2D.1 `applicant_info.gender`

Here we focus on `applicant_info.gender` because it is used downstream for bias/fairness analysis and must be standardized before analysis and reporting.


In [727]:
# Read gender as a string column so blanks/NA are handled consistently
g = df_raw["applicant_info.gender"].astype("string")

# Full raw value distribution
vc = g.value_counts(dropna=False)
vc_pct = (vc / n * 100).round(2)

# Combine counts and percentages into one table 
gender_table = pd.DataFrame({
    "count": vc,
    "%": vc_pct
})

print("Raw gender distribution (count and %):")
display(gender_table)

# Summary metric for shorthand and blank/missing values
n_M = vc.get("M", 0)
n_F = vc.get("F", 0)
n_blank = vc.get("", 0)
n_na = vc.get(pd.NA, 0) 

print("Total shorthand (M+F):", n_M + n_F, f"({(n_M+n_F)/n*100:.2f}%)")
print("Blank + NA:", n_blank + n_na, f"({(n_blank+n_na)/n*100:.2f}%)")

Raw gender distribution (count and %):


,count,%
applicant_info.gender,,
Male,195,38.84
Female,193,38.45
F,58,11.55
M,53,10.56
,2,0.4
<NA>,1,0.2


Total shorthand (M+F): 111 (22.11%)
Blank + NA: 3 (0.60%)


Results show that `applicant_info.gender` is inconsistently coded, mixing full labels (`Male`/`Female`) with shorthand codes (`M`/`F`). The shorthand values represent 22.11% of observations and there's also a small number of balnk/missing entries (0.60%). 

In the remediation, we will standardize gender coding by mapping `M → Male` and `F → Female`, and convert blank strings to missing to ensure a clean, consistent categorical field.

#### 2D.2 `decision.rejection_reason`

We also check `decision.rejection_reason`, which should behave as a categorical field for denied applications.

In [728]:
# Filter to denied applications and grab rejection_reason
rr = df_raw.loc[df_raw["decision.loan_approved"] == False, "decision.rejection_reason"].astype("string")

# Basic completeness + cardinality
print("Denied loans:", (df_raw["decision.loan_approved"] == False).sum())
print("Rejection reason non-missing:", rr.notna().sum())
print("Unique rejection reasons:", rr.dropna().nunique())

# Value distribution (counts + % among non-missing)
vc = rr.value_counts(dropna=True)
rr_table = pd.DataFrame({
    "count": vc,
    "pct_%": (vc / vc.sum() * 100).round(2)
})
display(rr_table)

# (Optional) quick check for messy formatting variants
# Shows any labels that differ only by casing/spacing once normalized
rr_norm = rr.str.strip().str.lower()
variant_check = (
    pd.DataFrame({"raw": rr.dropna(), "norm": rr_norm.dropna()})
    .groupby("norm")["raw"].nunique()
    .sort_values(ascending=False)
)

Denied loans: 210
Rejection reason non-missing: 210
Unique rejection reasons: 4


,count,pct_%
decision.rejection_reason,,
algorithm_risk_score,170,80.95
insufficient_credit_history,23,10.95
high_dti_ratio,13,6.19
low_income,4,1.9


Among denied applications, `decision.rejection_reason` is fully populated (210/210 non-missing) and behaves as a categorical variable with 4 distinct labels. No need for recoding here. 

### 2E. Inconsistent date formats

**Dimension:** Consistency  

In this step, we assess whether the two date fields (`applicant_info.date_of_birth` and `processing_timestamp`) are stored in a consistent way. Specifically, we verify if values in each column can be reliably parsed into datetimes.


In [729]:
dob = df_raw["applicant_info.date_of_birth"]
ts  = df_raw["processing_timestamp"]

# Try to parse the date fields (invalid formats become NaT)
dob_parsed = pd.to_datetime(dob, errors="coerce", utc=False)
ts_parsed  = pd.to_datetime(ts, errors="coerce", utc=True)

# Identify values that are present but failed to parse
invalid_dob = dob.notna() & dob_parsed.isna()
invalid_ts  = ts.notna()  & ts_parsed.isna()

print("DOB missing:", dob.isna().sum(), f"({dob.isna().mean()*100:.2f}%)")
print("DOB invalid format:", invalid_dob.sum(), f"({invalid_dob.mean()*100:.2f}%)")

print("processing_timestamp missing:", ts.isna().sum(), f"({ts.isna().mean()*100:.2f}%)")
print("processing_timestamp invalid format:", invalid_ts.sum(), f"({invalid_ts.mean()*100:.2f}%)")

DOB missing: 1 (0.20%)
DOB invalid format: 161 (32.07%)
processing_timestamp missing: 440 (87.65%)
processing_timestamp invalid format: 0 (0.00%)


We can observe that `applicant_info.date_of_birth` has a high rate of unparseable values (161 rows, 32.07%), indicating inconsistent date formats across records. 

On the other hand, `processing_timestamp` is largely missing (440 rows, 87.65%), but when present it is consistently parseable.

In [730]:
# Show a few examples where DOB is present but could not be parsed
invalid_dob_rows = df_raw[invalid_dob][["_id", "applicant_info.date_of_birth"]].head(15)
display(invalid_dob_rows)

,_id,applicant_info.date_of_birth
5,app_275,14/02/1982
6,app_099,28/01/1990
11,app_320,01/12/1978
14,app_307,1990/07/26
21,app_173,18/07/1979
23,app_289,20/04/1979
26,app_075,
32,app_274,1986/11/20
35,app_276,1995/05/07
36,app_386,03/20/1968


The sample of invalid DOB rows shows that `applicant_info.date_of_birth` is stored in mixed string formats: `DD/MM/YYYY`, `YYYY/MM/DD`, or `MM/DD/YYYY`

Because the column is not standardized to one format, `pd.to_datetime(..., errors="coerce")` fails to parse a significant share of records, which are then coerced to `NaT`. In remediation, we will standardize DOBs to a single canonical format (e.g., ISO `YYYY-MM-DD`) by parsing with explicit rules and treating truly unparseable values as missing.

In [731]:
# DOB format breakdown

# Clean DOB strings
dob = df_raw["applicant_info.date_of_birth"].astype("string").str.strip().replace({"": pd.NA})

# Detect common DOB patterns
iso = dob.str.match(r"^\d{4}[-/]\d{2}[-/]\d{2}$", na=False)      # YYYY-MM-DD or YYYY/MM/DD
slash = dob.str.match(r"^\d{2}/\d{2}/\d{4}$", na=False)          # ??/??/YYYY

# Flag ambiguous slash dates when both first numbers could be day or month
p1 = pd.to_numeric(dob.str.slice(0, 2), errors="coerce")
p2 = pd.to_numeric(dob.str.slice(3, 5), errors="coerce")
ambig = slash & (p1 <= 12) & (p2 <= 12)

# Build a summary table
dob_summary = pd.DataFrame({
    "count": [
        dob.isna().sum(),
        iso.sum(),
        slash.sum(),
        ambig.sum(),
        (dob.notna() & ~(iso | slash)).sum()
    ]
}, index=[
    "DOB missing",
    "DOB ISO-like (YYYY-MM-DD or YYYY/MM/DD)",
    "DOB slash (DD/MM/YYYY or MM/DD/YYYY)",
    "DOB ambiguous slash (both parts <= 12)",
    "DOB other format"
])
dob_summary["%"] = (dob_summary["count"] / len(dob) * 100).round(2)
display(dob_summary)

,count,%
DOB missing,5,1.00
DOB ISO-like (YYYY-MM-DD or YYYY/MM/DD),396,78.88
DOB slash (DD/MM/YYYY or MM/DD/YYYY),101,20.12
DOB ambiguous slash (both parts <= 12),39,7.77
DOB other format,0,0.00


As shown most `applicant_info.date_of_birth` values are in an ISO-style format (`YYYY-MM-DD` or `YYYY/MM/DD`), which is straightforward to parse consistently.  

A smaller share appears in a slash format (`??/??/YYYY`). This format can represent either `DD/MM/YYYY` or `MM/DD/YYYY`, and some values are ambiguous when both the day and month are ≤ 12.

In the remediation, since the dataset doesn't specify the locale/date convention, we assume the ambiguous DOB values follow `DD/MM/YYYY`, once is the most common in the EU. 

### 2F. Invalid/impossible values 

**Dimension:** Validity

#### 2F.1 Numeric fields

In this step we validate key numeric fields by checking for values that are outside plausible ranges, such as negative balances/months, or ratios outside (0,1).

In [732]:
# Convert numeric-like columns to numbers (strings become numeric; bad values become NaN)
income = pd.to_numeric(df_raw["financials.annual_income"], errors="coerce")
chm    = pd.to_numeric(df_raw["financials.credit_history_months"], errors="coerce")
dti    = pd.to_numeric(df_raw["financials.debt_to_income"], errors="coerce")
sav    = pd.to_numeric(df_raw["financials.savings_balance"], errors="coerce")
rate   = pd.to_numeric(df_raw["decision.interest_rate"], errors="coerce")
amt    = pd.to_numeric(df_raw["decision.approved_amount"], errors="coerce")

# Count values that violate basic business/validity rules
invalid = {
    "annual_income < 0": (income < 0).sum(),
    "credit_history_months < 0": (chm < 0).sum(),
    "debt_to_income < 0": (dti < 0).sum(),
    "debt_to_income > 1": (dti > 1).sum(),
    "savings_balance < 0": (sav < 0).sum(),
    "interest_rate < 0": (rate < 0).sum(),
    "interest_rate > 1": (rate > 1).sum(),  # if rate is stored as fraction; if it's % (e.g. 3.7), we'll adjust
    "approved_amount < 0": (amt < 0).sum(),
}

# Build a reporting table with counts + percentages (of all rows)
invalid_table = pd.DataFrame({
    "count": pd.Series(invalid),
    "pct_%": (pd.Series(invalid) / n * 100).round(2)
})

display(invalid_table)


,count,pct_%
annual_income < 0,0,0.00
credit_history_months < 0,2,0.40
debt_to_income < 0,0,0.00
debt_to_income > 1,1,0.20
savings_balance < 0,1,0.20
interest_rate < 0,0,0.00
interest_rate > 1,292,58.17
approved_amount < 0,0,0.00


According to the table, most numeric fields respect basic validity rules, but we do observe a few issues: `financials.credit_history_months` has two negative values, which is impossible, `financials.debt_to_income` has one value above 1 (outside the expected [0,1] ratio), and `financials.savings_balance` includes a negative value.

Additionally, `decision.interest_rate` appears “invalid” under a fraction-based rule (`> 1`). We will then check if the values represent percentages points instead of a [0,1] ratio:

In [733]:
rate = pd.to_numeric(df_raw["decision.interest_rate"], errors="coerce")

# Count rates outside a realistic percent range
invalid_rate_pct = ((rate < 0) | (rate > 100)).sum()
print("interest_rate outside [0, 100] percent:", invalid_rate_pct, f"({invalid_rate_pct/len(df_raw)*100:.2f}%)")

# Show min/max to justify treating this as percent points (not fractions)
print("interest_rate min/max (excluding NaN):", rate.min(), rate.max())

interest_rate outside [0, 100] percent: 0 (0.00%)
interest_rate min/max (excluding NaN): 2.5 6.5


The observed range (minimum 2.5 and maximum 6.5) indicates it is stored in percentage points, meaning 3.7 is 3.7%, so `> 1` is not treated as invalid. We validate interest rates using a percent range (0–100) instead.

#### 2F.2 Identifier-like fields
##### SSN, ZIP, and Ip address

We validate identifier-like fields against basic format rules: 
- SSN should match `###-##-####`
- ZIP code should be exactly 5 digits
- IP address should be a valid IPv4 address (`###.###.###.###`). 

In [734]:
# Read the three fields as strings
ssn  = df_raw["applicant_info.ssn"].astype("string").str.strip()
zipc = df_raw["applicant_info.zip_code"].astype("string").str.strip()
ip   = df_raw["applicant_info.ip_address"].astype("string").str.strip()

# SSN pattern ###-##-####
ssn_invalid = ssn.dropna().str.match(r"^\d{3}-\d{2}-\d{4}$") == False
print("Invalid SSN format:", ssn_invalid.sum(), f"({ssn_invalid.sum()/len(df_raw)*100:.2f}%)")

# ZIP 5 digits (string)
zip_invalid = zipc.dropna().str.match(r"^\d{5}$") == False
print("Invalid ZIP format:", zip_invalid.sum(), f"({zip_invalid.sum()/len(df_raw)*100:.2f}%)")

# IP valid IPv4
def is_valid_ip(x):
    if pd.isna(x):
        return True
    try:
        ipaddress.ip_address(str(x))
        return True
    except ValueError:
        return False

ip_invalid = ip.dropna().map(lambda x: not is_valid_ip(x))
print("Invalid IP address:", ip_invalid.sum(), f"({ip_invalid.sum()/len(df_raw)*100:.2f}%)")

Invalid SSN format: 0 (0.00%)
Invalid ZIP format: 1 (0.20%)
Invalid IP address: 0 (0.00%)


In [735]:
# Show rows where ZIP is present but not 5 digits
zipc = df_raw["applicant_info.zip_code"].astype("string")
bad_zip_mask = zipc.notna() & (zipc.str.match(r"^\d{5}$") == False)

display(df_raw.loc[bad_zip_mask, ["_id", "applicant_info.zip_code", "applicant_info.ssn", "applicant_info.full_name"]])

,_id,applicant_info.zip_code,applicant_info.ssn,applicant_info.full_name
26,app_075,,NaN,Margaret Williams


SSN and IP formats are fully valid in the dataset.  

We found 1 invalid ZIP (0.20%), which is a blank string. We will convert it to missing in the remediation.   

##### Email 

Now we check `applicant_info.email` to see if the value contains an `@`, and the domain part contains at least one dot.

In [736]:
# Clean email column: cast to string, trim spaces, turn "" into missing
email = df_raw["applicant_info.email"].astype("string").str.strip()
email = email.replace({"": pd.NA})

# Basic structure check: must contain "@"
has_at = email.notna() & email.str.contains("@", na=False)

# Extract domain (everything after "@") for rows that have "@"
domain = email.where(has_at).str.split("@", n=1).str[1].str.lower().str.strip()

# Define invalid cases we want to count
missing_at  = email.notna() & ~has_at                              # no "@"
bad_domain  = has_at & (~domain.str.contains(r"\.", na=False))     # domain has no dot
empty_email = email.isna()

print("Emails missing:", int(empty_email.sum()), f"({empty_email.mean()*100:.2f}%)")
print("Missing/invalid domain (no @):", int(missing_at.sum()), f"({missing_at.mean()*100:.2f}%)")
print("Domain bad format (no dot):", int(bad_domain.sum()), f"({bad_domain.mean()*100:.2f}%)")

# Show examples of invalid cases
display(df_raw.loc[missing_at | bad_domain, ["_id", "applicant_info.email"]].head(15))

# Domain counts (valid only)
valid_domain = has_at & ~bad_domain
display(domain[valid_domain].value_counts().head(15))

Emails missing: 7 (1.39%)
Missing/invalid domain (no @): 1 (0.20%)
Domain bad format (no dot): 2 (0.40%)


,_id,applicant_info.email
181,app_299,test.user.outlook.com
276,app_068,john.doe@invalid
369,app_146,sarah.smith@


applicant_info.email
gmail.com         72
yahoo.com         70
mail.com          68
outlook.com       68
protonmail.com    61
hotmail.com       56
aol.com           50
icloud.com        47
Name: count, dtype: int64

Most emails follow a valid-looking structure, but a small number fail basic formatting checks: 
- 7 emails are missing (1.39%)
- 1 value is missing `@` (0.20%)
- 2 values have a domain with no dot (0.40%).

To remediate this, we convert blank strings to missing (`NA`) and treat emails that miss `@` or have invalid domain as missing rather than guessing or correcting them.

### 2G. Cross-field consistency checks 

**Dimension:** Accuracy

We verify that related decision fields are consistent:
- Approved loans should have `approved_amount` and `interest_rate`, and should not have `rejection_reason`.
- Denied loans should have `rejection_reason`, and should not have approval terms.

In [737]:
approved = df_raw["decision.loan_approved"] == True
denied   = df_raw["decision.loan_approved"] == False

# Approved but missing terms
approved_missing_terms = approved & (
    df_raw["decision.approved_amount"].isna() | df_raw["decision.interest_rate"].isna()
)

# Approved but has rejection reason
approved_has_rejection = approved & df_raw["decision.rejection_reason"].notna()

# Denied but missing rejection reason
denied_missing_rejection = denied & df_raw["decision.rejection_reason"].isna()

# Denied but has terms
denied_has_terms = denied & (
    df_raw["decision.approved_amount"].notna() | df_raw["decision.interest_rate"].notna()
)

print("Approved but missing approved_amount or interest_rate:", int(approved_missing_terms.sum()))
print("Approved but has rejection_reason:", int(approved_has_rejection.sum()))
print("Denied but missing rejection_reason:", int(denied_missing_rejection.sum()))
print("Denied but has approved_amount or interest_rate:", int(denied_has_terms.sum()))

Approved but missing approved_amount or interest_rate: 0
Approved but has rejection_reason: 0
Denied but missing rejection_reason: 0
Denied but has approved_amount or interest_rate: 0


No inconsistencies were found.

### 2H. Key findings

---
## 3. Remediation: Cleaning pipeline
---

In this section we apply the cleaning steps needed to address the issues identified in the data quality checks. The goal is to produce a consistent, analysis-ready dataset while preserving as much information as possible. 

In general, we apply field-level fixes (standardizing formats and setting invalid values to `NA`) and only remove records when necessary.

### 3A. Remove duplicate application IDs

To work with one row per application, we first resolve duplicated `_id` values. When the same `_id` appears more than once, we keep the version of the record that is more complete (the one with more filled-in fields) and drop the extra duplicate rows. This gives us a single, best representative record for each application.

In [738]:
# Start remediation from the raw dataframe
df_clean = df_raw.copy()

# Count non-missing values across all columns
df_clean["_non_missing"] = df_clean.notna().sum(axis=1)

# Sort so the most complete row per _id comes first
df_clean = df_clean.sort_values(["_id", "_non_missing"], ascending=[True, False])

# Keep one row per _id
df_clean = df_clean.drop_duplicates(subset=["_id"], keep="first").drop(columns=["_non_missing"])

# Quick check
print("Duplicate _id after remediation:", int(df_clean["_id"].duplicated().sum()))
print("Rows after _id dedupe:", len(df_clean))

Duplicate _id after remediation: 0
Rows after _id dedupe: 500


### 3B. Flag duplicate SSNs

We don't automatically drop duplicate SSNs. Since the records have different applicant names, we treat these as identifier conflicts and flag them for review. In the cleaned dataset, we keep both rows but mark ssn_is_duplicate = True and set the SSN to missing so it isn't used as a reliable identifier.

In [739]:
# Flag SSNs that appear in more than one row, ignoring missing SSNs
ssn = df_clean["applicant_info.ssn"]

df_clean["ssn_is_duplicate"] = ssn.notna() & ssn.duplicated(keep=False)

# Set SSN to missing for duplicated SSNs 
df_clean.loc[df_clean["ssn_is_duplicate"], "applicant_info.ssn"] = pd.NA

# Quick check and show examples
print("Rows with duplicate SSN:", int(df_clean["ssn_is_duplicate"].sum()))

display(
    df_clean.loc[
        df_clean["ssn_is_duplicate"],
        ["_id", "applicant_info.ssn", "applicant_info.full_name", "ssn_is_duplicate"]
    ].head(10)
)

Rows with duplicate SSN: 4


,_id,applicant_info.ssn,applicant_info.full_name,ssn_is_duplicate
122,app_016,<NA>,Gary Wilson,True
92,app_088,<NA>,Susan Martinez,True
16,app_101,<NA>,Sandra Smith,True
499,app_234,<NA>,Samuel Hill,True


### 3C. Convert blank strings to missing values

Some text fields contain blank strings (`""` or `"   "`) instead of true missing values. To handle missingness consistently across the dataset, we trim whitespace and convert blank strings to `NA` for all text-like columns.

In [740]:
# Select text-like columns
text_cols = df_clean.select_dtypes(include=["object", "string"]).columns

# Trim whitespace and convert "" to NA
df_clean[text_cols] = df_clean[text_cols].astype("string").apply(lambda s: s.str.strip())
df_clean[text_cols] = df_clean[text_cols].replace({"": pd.NA})

# Quick check: number of blank strings remaining
blank_remaining = df_clean[text_cols].apply(lambda s: s.astype("string").str.strip().eq("").sum()).sum()
print("Total blank strings remaining:", int(blank_remaining))

Total blank strings remaining: 0


### 3D. Unify yearly income

The dataset stores the same concept (yearly income) in two different columns: `financials.annual_income` and `financials.annual_salary`. 
We first convert both columns to numeric (solving the issue with string entries in the variable), then fill missing `annual_income` with `annual_salary` and drop `annual_salary` to keep one canonical income field.

In [741]:
# Convert both columns to numeric
df_clean["financials.annual_income"] = pd.to_numeric(df_clean["financials.annual_income"], errors="coerce")
df_clean["financials.annual_salary"] = pd.to_numeric(df_clean["financials.annual_salary"], errors="coerce")

# Fill missing annual_income with annual_salary
df_clean["financials.annual_income"] = df_clean["financials.annual_income"].fillna(df_clean["financials.annual_salary"])

# Drop annual_salary to keep one canonical income field
df_clean = df_clean.drop(columns=["financials.annual_salary"])

# Quick check: remaining missing annual_income + confirm salary is gone
print("Missing annual_income:", int(df_clean["financials.annual_income"].isna().sum()))
print("annual_salary column exists?", "financials.annual_salary" in df_clean.columns)

Missing annual_income: 0
annual_salary column exists? False


### 3E. Standardize gender coding

To keep categorical analysis consistent, we standardize `applicant_info.gender` so shorthand codes (`M`/`F`) are mapped to the full labels (`Male`/`Female`). Any remaining blanks are already handled as `NA`.

In [742]:
# Standardize shorthand to full labels
df_clean["applicant_info.gender"] = (
    df_clean["applicant_info.gender"]
    .astype("string")
    .replace({"M": "Male", "F": "Female"})
)

# Quick check (after): value counts
display(df_clean["applicant_info.gender"].value_counts(dropna=False))

applicant_info.gender
Female    251
Male      247
<NA>        2
Name: count, dtype: Int64

### 3F. Standardize date of birth format

`applicant_info.date_of_birth` appears in multiple formats. To make the field consistent, we parse the values and store them in a single standard format: `DD-MM-YYYY`. For slash-formatted dates where day/month order isn't explicit, we assume the `DD/MM/YYYY` format, since it's the most common convention in Europe and the dataset doesn't provide any data indicating a location or date standard.

In [743]:
# Clean raw DOB strings
dob_raw = (
    df_clean["applicant_info.date_of_birth"]
    .astype("string")
    .str.strip()
    .replace({"": pd.NA})
)

# Start empty datetime series
dob_dt = pd.Series(pd.NaT, index=df_clean.index)

# YYYY-MM-DD
m = dob_raw.notna() & dob_raw.str.match(r"^\d{4}-\d{2}-\d{2}$", na=False)
dob_dt.loc[m] = pd.to_datetime(dob_raw.loc[m], errors="coerce", format="%Y-%m-%d")

# YYYY/MM/DD
m = dob_raw.notna() & dob_raw.str.match(r"^\d{4}/\d{2}/\d{2}$", na=False)
dob_dt.loc[m] = pd.to_datetime(dob_raw.loc[m], errors="coerce", format="%Y/%m/%d")

# Slash format: try DD/MM/YYYY first (assumption)
m_slash = dob_raw.notna() & dob_raw.str.match(r"^\d{2}/\d{2}/\d{4}$", na=False)
dob_dt.loc[m_slash] = pd.to_datetime(dob_raw.loc[m_slash], errors="coerce", format="%d/%m/%Y")

# Fallback: MM/DD/YYYY only for the ones DD/MM couldn't parse
m_mdy = m_slash & dob_dt.isna()
dob_dt.loc[m_mdy] = pd.to_datetime(dob_raw.loc[m_mdy], errors="coerce", format="%m/%d/%Y")

# Store standardized string format (DD-MM-YYYY)
df_clean["applicant_info.date_of_birth"] = dob_dt.dt.strftime("%d-%m-%Y").astype("string")

# Quick check
still_bad = dob_raw.notna() & dob_dt.isna()
print("Remaining unparseable DOB:", int(still_bad.sum()))

# Show examples of standardized (valid) DOBs
good_dob = df_clean["applicant_info.date_of_birth"].notna()

print("Standardized DOB (non-missing):", int(good_dob.sum()))
display(df_clean.loc[good_dob, ["_id", "applicant_info.date_of_birth"]].head(10))


Remaining unparseable DOB: 0
Standardized DOB (non-missing): 496


,_id,applicant_info.date_of_birth
383,app_001,27-05-1986
339,app_002,01-08-1999
284,app_003,24-08-1982
255,app_004,28-02-1995
136,app_005,19-06-1960
328,app_006,12-07-1987
12,app_007,13-06-1989
81,app_008,12-07-1993
390,app_009,20-11-1989
47,app_010,07-09-1996


### 3G. Fixing invalid / impossible values

#### 3G.1 Impossible numeric values

We correct the impossible numeric values (negative months, negative savings balances, and a debt-to-income ratio above 1). Instead of dropping whole rows, we set those specific values to missing (NA) so the record can still be used.

In [744]:
# Coerce to numeric
chm = pd.to_numeric(df_clean["financials.credit_history_months"], errors="coerce")
dti = pd.to_numeric(df_clean["financials.debt_to_income"], errors="coerce")
sav = pd.to_numeric(df_clean["financials.savings_balance"], errors="coerce")

# Masks for impossible values
bad_chm = chm < 0
bad_dti = dti > 1
bad_sav = sav < 0

# Apply fixes (set to missing)
df_clean.loc[bad_chm, "financials.credit_history_months"] = pd.NA
df_clean.loc[bad_dti, "financials.debt_to_income"] = pd.NA
df_clean.loc[bad_sav, "financials.savings_balance"] = pd.NA

# Quick checks (should be 0 after remediation)
print("credit_history_months < 0:", int(pd.to_numeric(df_clean["financials.credit_history_months"], errors="coerce").lt(0).sum()))
print("debt_to_income > 1:", int(pd.to_numeric(df_clean["financials.debt_to_income"], errors="coerce").gt(1).sum()))
print("savings_balance < 0:", int(pd.to_numeric(df_clean["financials.savings_balance"], errors="coerce").lt(0).sum()))

credit_history_months < 0: 0
debt_to_income > 1: 0
savings_balance < 0: 0


#### 3G.2 Invalid emails

To keep contact fields consistent, we treat emails as valid only when they contain an `@` and the domain contains a dot. The remaining are set to missing (`NA`).

In [745]:
# Clean emails
email = df_clean["applicant_info.email"].astype("string").str.strip()
has_at = email.notna() & email.str.contains("@", na=False)

# Domain = part after "@"
domain = email.where(has_at).str.split("@", n=1).str[1].str.lower().str.strip()

# Invalid cases
missing_at = email.notna() & ~has_at
bad_domain = has_at & (~domain.str.contains(r"\.", na=False))

# Quick check (before)
print("Invalid email (missing @):", int(missing_at.sum()))
print("Invalid email (bad domain):", int(bad_domain.sum()))

# Set invalid emails to missing
df_clean.loc[missing_at | bad_domain, "applicant_info.email"] = pd.NA

# Quick check (after)
email_after = df_clean["applicant_info.email"].astype("string")
print("Remaining invalid emails:", int((email_after.notna() & ~email_after.str.contains("@", na=False)).sum()))

Invalid email (missing @): 1
Invalid email (bad domain): 2
Remaining invalid emails: 0
